In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [24]:
import sys
sys.path

['/Users/tim/PycharmProjects/HOI/examples',
 '/Users/tim/opt/anaconda3/lib/python39.zip',
 '/Users/tim/opt/anaconda3/lib/python3.9',
 '/Users/tim/opt/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/tim/opt/anaconda3/lib/python3.9/site-packages',
 '/Users/tim/opt/anaconda3/lib/python3.9/site-packages/aeosa',
 '/Users/Tim/PycharmProjects/HOI/',
 '/Users/Tim/PycharmProjects/HOI/']

In [25]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from ipywidgets import IntProgress

In [26]:
from synthetic_data import stationary_pb_ts
from HOI.preprocessings import compute_kernel
from HOI.tests import test_independence

In [32]:
# all pairwise independent but jointly dependent
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in tqdm(np.arange(100)):
        df = stationary_pb_ts(t_time = 1200, d = d, mode = "case1", a=0.5)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']],
                                         [data_dict['d1'], data_dict['d2'], data_dict['d3']],
                                          mode = 'stat_ts', n_perms=5000, alpha=0.05)
        rejects = rejects + reject
    power[str(d)] = rejects/300

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [30]:
power

{'0.1': 0.34,
 '0.2': 0.31,
 '0.30000000000000004': 0.47,
 '0.4': 0.49,
 '0.5': 0.62,
 '0.6': 0.71,
 '0.7000000000000001': 0.72,
 '0.8': 0.79,
 '0.9': 0.87}

In [ ]:
# 100% better than 3-way HSIC, comparable with lancaster test from paulr

In [ ]:
 # 2 pairwise dependent, xy pairwise independent and jointly dependent
power ={}
for d in tqdm(np.arange(0.1, 1, 0.1)):
    rejects = 0
    for i in tqdm(np.arange(100)):
        df = stationary_pb_ts(n_sample = 500, d = 0.65, mode = "case2", a=0.5)
        data_dict, kernel_dict = compute_kernel(df)
        _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']],
                                         [data_dict['d1'], data_dict['d2'], data_dict['d3']],
                                          mode = 'stat_ts', n_perms=5000, alpha=0.05)
        _, _, reject = test_independence([kernel_dict['d1'], kernel_dict['d3']],
                                         [data_dict['d1'], data_dict['d3']],
                                          mode = 'stat_ts', n_perms=5000, alpha=0.05)
        _, _, reject = test_independence([kernel_dict['d2'], kernel_dict['d3']],
                                         [data_dict['d2'], data_dict['d3']],
                                          mode = 'stat_ts', n_perms=5000, alpha=0.05)
        rejects = rejects + reject
    power[str(d)] = rejects/100

In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
from synthetic_data import stationary_pb_ts
from HOI.preprocessings import compute_kernel
from HOI.tests import test_independence

In [16]:
import statsmodels.api as sm

In [18]:
# all pairwise independent but jointly dependent
df = stationary_pb_ts(t_time = 300, seed = 11, d = 0.5, mode = "case1", a=0.5)

In [29]:
data_dict, kernel_dict = compute_kernel(df)

In [30]:
data_list = [data_dict['d1'], data_dict['d2'], data_dict['d3']]

In [31]:
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [32]:
test_independence(k_list=k_list, data_list=data_list, mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0026235365848013004, 0.0017002146919464333, 1)

In [33]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d2']], 
                  data_list=[data_dict['d1'], data_dict['d2']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0009016972028506043, 0.0018298917154126778, 0)

In [34]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d3']], 
                  data_list=[data_dict['d1'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0006952669977832171, 0.0019504181366873374, 0)

In [35]:
test_independence(k_list=[kernel_dict['d2'], kernel_dict['d3']], 
                  data_list=[data_dict['d2'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0003036424696591933, 0.001997042274738181, 0)

In [110]:
# 2 pairwise dependent, xy pairwise independent and jointly dependent
df = stationary_pb_ts(n_sample = 500, seed = 109, d = 0.65, mode = "case2", a=0.5)
data_dict, kernel_dict = compute_kernel(df)
data_list = [data_dict['d1'], data_dict['d2'], data_dict['d3']]
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [111]:
test_independence(k_list=k_list, data_list=data_list, mode='stat_ts', n_perms=1000, alpha=0.05)

(0.016044540601538315, 0.009292483895777054, 1)

In [112]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d2']], 
                  data_list=[data_dict['d1'], data_dict['d2']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0005067694109530674, 0.0027047636288386467, 0)

In [113]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d3']], 
                  data_list=[data_dict['d1'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.010298398467857739, 0.00834964775135938, 1)

In [114]:
test_independence(k_list=[kernel_dict['d2'], kernel_dict['d3']], 
                  data_list=[data_dict['d2'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.013573335902312134, 0.00862558261305904, 1)

In [78]:
# independent
df = stationary_pb_ts(n_sample = 500, seed = 190, d = 1.1, mode = "case3", a=0.5)
data_dict, kernel_dict = compute_kernel(df)
data_list = [data_dict['d1'], data_dict['d2'], data_dict['d3']]
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [79]:
test_independence(k_list=k_list, data_list=data_list, mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0009576279285979372, 0.002181602178572525, 0)

In [80]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d2']], 
                  data_list=[data_dict['d1'], data_dict['d2']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0005155378666643617, 0.001913283012410005, 0)

In [81]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d3']], 
                  data_list=[data_dict['d1'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.000368500396157212, 0.001722822310883737, 0)

In [82]:
test_independence(k_list=[kernel_dict['d2'], kernel_dict['d3']], 
                  data_list=[data_dict['d2'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0004885169485012719, 0.0023070856703295726, 0)

In [36]:
# independent
df = stationary_pb_ts(n_sample = 500, seed = 190, d = 0.7, mode = "case1", a=1)
data_dict, kernel_dict = compute_kernel(df)
data_list = [data_dict['d1'], data_dict['d2'], data_dict['d3']]
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [37]:
test_independence(k_list=k_list, data_list=data_list, mode='iid', n_perms=1000, alpha=0.05)

(0.022273081846982956, 0.0013290053004219593, 1)

In [39]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d2']], 
                  data_list=[data_dict['d1'], data_dict['d2']], 
                  mode='iid', n_perms=1000, alpha=0.05)

(0.007686663840210994, 0.0008816469741782518, 1)

In [40]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d3']], 
                  data_list=[data_dict['d1'], data_dict['d3']], 
                  mode='iid', n_perms=1000, alpha=0.05)

(0.0020899646776492142, 0.0009328691440982917, 1)

In [41]:
test_independence(k_list=[kernel_dict['d2'], kernel_dict['d3']], 
                  data_list=[data_dict['d2'], data_dict['d3']], 
                  mode='iid', n_perms=1000, alpha=0.05)

(0.026592307158503026, 0.0008846596143171626, 1)

In [6]:
x_mat = []
x = np.zeros(10)
y = np.ones(10)
x_mat.append(x)
x_mat.append(y)

In [9]:
np.array(x_mat).shape

(2, 10)

In [34]:
from sklearn.metrics import pairwise_kernels, pairwise_distances

In [ ]:
kernel_matrix = pairwise_kernels(data, metric='rbf', gamma=0.5 / (width(data) ** 2)

In [19]:
data_dict = df.to_dict('list')

In [28]:
data = np.array(data_dict['d1'])

In [29]:
data.shape

(300,)

In [32]:
x_mat = []
x_mat.append(data)
x_mat.append(data)
x_mat.append(data)
np.array(x_mat).shape

(3, 300)

In [35]:
dist_mat = pairwise_distances(x_mat, metric='euclidean')
width_Z = np.median(dist_mat[dist_mat > 0])

In [36]:
kernel_matrix = pairwise_kernels(x_mat, metric='rbf', gamma=0.5 / (width_Z ** 2))

In [37]:
kernel_matrix

array([[0.60653066, 0.60653066, 0.60653066],
       [0.60653066, 0.60653066, 0.60653066],
       [0.60653066, 0.60653066, 0.60653066]])